### KNN

Implemantação do classificador utilizando KNN.

In [1]:
import nltk
import itertools

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from string import ascii_lowercase

from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline

from sklearn.metrics import classification_report

from pymongo import MongoClient

%matplotlib inline

In [2]:
'''
Importação dos dados da base
'''
client = MongoClient()
client = MongoClient('localhost', 27800)
db = client.network_logs
db_unique_hostnames = db.unique_hostnames

df_hostnames = pd.DataFrame(list(db_unique_hostnames.find({})))

print(len(df_hostnames))
df_hostnames.head()

24208


,_id,catdesc,category,hostname,normalized_hostname,normalized_hostname_url,normalized_url,service,url
0,59a1e31988b032071ef8c495,Information Technology,Information Technology,sb.scorecardresearch.com,scorecardresearch,scorecardresearch,,HTTPS,/
1,59a1e31a88b032071ef8c496,Information Technology,Information Technology,test.com,test,testpartneruidUPcbedebee,partneruidUPcbedebee,HTTP,/?partner_uid=UP47c5b905-0e1d-11e7-b092-00163e...
2,59a1e31a88b032071ef8c497,Information Technology,Information Technology,nexus-websocket-b.intercom.io,nexuswebsocketbintercom,nexuswebsocketbintercom,,HTTPS,/
3,59a1e31a88b032071ef8c498,Malicious Websites,Malicious or Illegal,aorta.clickagy.com,aortaclickagy,aortaclickagypixelgifadvertiseridgtqxdqpglista...,pixelgifadvertiseridgtqxdqpglistaedjpnobncchcm...,HTTP,/pixel.gif?advertiser_id=1gt8qx97dqpg&list=11a...
4,59a1e31a88b032071ef8c499,Information Technology,Information Technology,usersync.videoamp.com,usersyncvideoamp,usersyncvideoampusersyncpartneridpartneruserid...,usersyncpartneridpartneruseridredirecthttpsync...,HTTP,/usersync?partner_id=5797361&partner_user_id=4...


In [3]:
'''
Criação do vocabulário
'''
keywords = [''.join(trig) for trig in itertools.product(ascii_lowercase, repeat = 3)]
len(keywords)

17576

### Separação do conjunto

In [4]:
'''
Separação do conjunto de treinamento e validação
'''
url_train,url_test,label_train,label_test = train_test_split(df_hostnames['normalized_hostname'], df_hostnames['category'], test_size=0.3)

### Criação do pipeline

In [5]:
pipeline = Pipeline([
    ('wordsVector', CountVectorizer(analyzer='char', ngram_range=(3,3), vocabulary=keywords)),
    ('tf', TfidfTransformer(use_idf=False)),
    ('classifier', KNeighborsClassifier(n_neighbors=1))
])

In [6]:
pipeline.fit(url_train, label_train)

Pipeline(memory=None,
     steps=[('wordsVector', CountVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(3, 3), preprocessor=None, stop_words=None,
      ...owski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform'))])

In [7]:
predictions = pipeline.predict(url_test)

### Relatório da classificação

In [8]:
print(classification_report(label_test, predictions))

                                 precision    recall  f1-score   support

                          Adult       0.13      0.11      0.12        18
                    Advertising       0.59      0.53      0.56       290
               Arts and Culture       0.44      0.20      0.28        40
                       Business       0.55      0.32      0.40       739
                Content Servers       0.75      0.70      0.72       360
                 Domain Parking       0.25      0.11      0.15        18
                      Education       0.56      0.36      0.44       226
                  Entertainment       0.42      0.24      0.31       127
       File Sharing and Storage       0.87      0.82      0.85       153
            Finance and Banking       0.55      0.38      0.45        58
Freeware and Software Downloads       0.69      0.38      0.49        63
                          Games       0.70      0.63      0.66       179
            Health and Wellness       0.41      0.